In [20]:
%pip install pyscisci networkx python-Levenshtein sparse_dot_topn matplotlib boto3 widgetsnbextension ipywidgets "dask[distributed]" --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 62.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.49
    Uninstalling botocore-1.34.49:
      Successfully uninstalled botocore-1.34.49
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.49
    Uninstalling boto3-1.34.49:
      Successfully uninstalled boto3-1.34.49

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyscisci.all as pyscisci

import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

import dask.dataframe as dd

%matplotlib inline

path2openalex = '/disks/qnap2/data/t-yatima/openalex231124/data/'

DATA_DIR = '/disks/qnap2/data/t-yatima/scisciJP2024_tutorial/'
OUTPUT_WORKS_JSON_PKL = DATA_DIR + 'random_works_json.pkl'
OUTPUT_WORKS_DF_PKL = DATA_DIR + 'random_works_df.pkl'
OUTPUT_WORKS_REL_REF_DF_PKL = DATA_DIR + 'random_rel_ref_df.pkl'

myoa = pyscisci.OpenAlex(path2openalex, database_extension='csv.gz', keep_in_memory=True, show_progress=True, enable_dask=True) 

from dask.diagnostics import ProgressBar
ProgressBar().register()

from dask.distributed import progress

In [5]:
all_pub_ddf = myoa.load_publications()
all_pub_ddf

Loading Publications:   0%|          | 0/480 [00:00<?, ?it/s]

,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText
npartitions=480,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [7]:
# 9min
references_df = myoa.load_references()
display(references_df)

Loading pub2ref:   0%|          | 0/480 [00:00<?, ?it/s]

,CitingPublicationId,CitedPublicationId
0,3117235547,55183993
1,3117235547,657382535
2,3117235547,1540739244
3,3117235547,1582266453
4,3117235547,1642177666
...,...,...
9436475,2134809571,2090594024
9436476,2134809571,2124054840
9436477,2134809571,2147932722
9436478,2134809571,2158145285


In [30]:
!pip show pyscisci

Name: pyscisci
Version: 0.8.11
Summary: Science of Science
Home-page: https://github.com/ajgates42/pyscisci
Author: Alex Gates <ajgates42@gmail.com>
Author-email: 
License: MIT
Location: /disks/qnap2/notebook/t-yatima/.venv/lib/python3.10/site-packages
Requires: dask, lxml, nameparser, numba, numpy, pandas, requests, scikit-learn, scipy, tqdm, unidecode
Required-by: 


In [6]:
# 8min
random_works_ddf = all_pub_ddf.sample(frac=0.0001, random_state=0)
random_works_df = random_works_ddf.compute()

[########################################] | 100% Completed | 502.18 s


In [9]:
random_works_df

,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText
99910,2289614885,NaN,2015,0,On the margin : technology helps carriers choo...,2015-08-01,article,NaN,NaN,NaN,NaN,NaN,NaN,False,False
281133,588250911,NaN,2008,0,MISCELLANEOUS 日本語表現能力に関する実践的教育-演習「プロゼミII」の実態調査-,2008-03-18,article,NaN,NaN,2,NaN,94,104,False,False
18677,1997677555,NaN,2009,1,Errors of target detecting using triangulation...,2009-10-23,article,NaN,NaN,NaN,NaN,413,414,False,False
300106,641542497,NaN,2011,0,Mellan nation och omvärld,2011-01-01,article,NaN,NaN,NaN,NaN,297,301,False,False
87096,2252829188,NaN,1998,0,"解决砌块建筑“裂,热,漏”问题的探讨",1998-01-01,article,NaN,NaN,NaN,12,18,18,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267920,2039546701,NaN,1993,26,Two-Dimensional Contact-Type Image Sensor Usin...,1993-01-01,article,10.1143/jjap.32.458,NaN,32,1S,458,458,False,False
33204,2044872246,NaN,2001,31,Photoelectrochemistry of Langmuir−Blodgett Fil...,2001-09-07,article,10.1021/jp010250h,NaN,105,39,9406,9412,False,False
22768,1973947117,NaN,2006,31,Strain relaxation and surface morphology of ni...,2006-03-01,article,10.1016/j.susc.2005.12.038,NaN,600,5,1099,1106,False,False
69355,1999900463,NaN,1995,30,Uniform-size hydrophobic polymer-based separat...,1995-01-01,article,10.1016/0021-9673(94)00899-k,7881539,690,1,21,28,False,False


In [25]:
# 11min
all_paa = myoa.load_publicationauthoraffiliation()
all_paa

Loading Publication Author Affiliation:   0%|          | 0/480 [00:00<?, ?it/s]

/home/t-yatima/t-yatima/.venv/lib/python3.10/site-packages/pyscisci/datasource/readwrite.py:203: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat(data)


,PublicationId,AuthorId,AffiliationId,AuthorSequence,OrigAuthorName
0,4287754650,5062226449,NaN,1,Enrico Biffis
1,4287754650,5033140346,NaN,2,Erik Chavez
2,4287754650,5061623148,NaN,3,Alexis Louaas
3,4287754650,5074952038,NaN,4,Pierre M. Picard
4,4287938432,5045351019,NaN,1,Arthur Hugeat
...,...,...,...,...,...
1810902,2134809571,5004197195,1.355989e+08,3,Hiroki Shibata
1810903,2134809571,5061432587,4.210123e+09,4,Tomomi Saito
1810904,2134809571,5029531503,4.240550e+07,5,Osamu Doi
1810905,2134809571,5004358248,4.240550e+07,6,Yutaka Ohya


In [32]:
# all_paaでPublicationIdごとにAuthorIdの数を数える
all_paa.drop(columns=['AffiliationId', 'AuthorSequence', 'OrigAuthorName'], inplace=True)
author_count = all_paa.groupby('PublicationId').count().reset_index()
author_count.columns = ['PublicationId', 'AuthorCount']  # 列名を変更

# random_works_dfに結果を結合する
random_works_df = random_works_df.merge(author_count, on='PublicationId', how='left')
random_works_df

,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText,AuthorCount
0,2289614885,NaN,2015,0,On the margin : technology helps carriers choo...,2015-08-01,article,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1.0
1,588250911,NaN,2008,0,MISCELLANEOUS 日本語表現能力に関する実践的教育-演習「プロゼミII」の実態調査-,2008-03-18,article,NaN,NaN,2,NaN,94,104,False,False,1.0
2,1997677555,NaN,2009,1,Errors of target detecting using triangulation...,2009-10-23,article,NaN,NaN,NaN,NaN,413,414,False,False,1.0
3,641542497,NaN,2011,0,Mellan nation och omvärld,2011-01-01,article,NaN,NaN,NaN,NaN,297,301,False,False,1.0
4,2252829188,NaN,1998,0,"解决砌块建筑“裂,热,漏”问题的探讨",1998-01-01,article,NaN,NaN,NaN,12,18,18,False,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22765,2039546701,NaN,1993,26,Two-Dimensional Contact-Type Image Sensor Usin...,1993-01-01,article,10.1143/jjap.32.458,NaN,32,1S,458,458,False,False,3.0
22766,2044872246,NaN,2001,31,Photoelectrochemistry of Langmuir−Blodgett Fil...,2001-09-07,article,10.1021/jp010250h,NaN,105,39,9406,9412,False,False,2.0
22767,1973947117,NaN,2006,31,Strain relaxation and surface morphology of ni...,2006-03-01,article,10.1016/j.susc.2005.12.038,NaN,600,5,1099,1106,False,False,7.0
22768,1999900463,NaN,1995,30,Uniform-size hydrophobic polymer-based separat...,1995-01-01,article,10.1016/0021-9673(94)00899-k,7881539,690,1,21,28,False,False,7.0


In [14]:
bibmet_id_list = [int(work) for work in random_works_df['PublicationId'].to_list()]
bibmet_id_list

[2289614885,
 588250911,
 1997677555,
 641542497,
 2252829188,
 2512095229,
 2409947584,
 3081744184,
 2938687893,
 390021029,
 4214577364,
 2034450803,
 2350122719,
 1980331161,
 949007422,
 909231072,
 760532667,
 1819722246,
 2118511552,
 1592133559,
 2595813847,
 2386915843,
 1551769421,
 428327019,
 3211327119,
 3199422493,
 2513379591,
 411846878,
 376426975,
 2354954625,
 2271864853,
 6189665,
 2904877874,
 2184584010,
 2384689382,
 154837109,
 3176903894,
 2350729102,
 2606918120,
 1979057808,
 2115242963,
 1926427285,
 2109699317,
 4283380004,
 1575580130,
 1969315279,
 2735098700,
 1535508539,
 644373152,
 2293877789,
 2121547664,
 2306391979,
 2064455131,
 2469548427,
 2241919256,
 1572194604,
 2377029689,
 2332771914,
 2769830383,
 2054779506,
 585882067,
 2581450158,
 1816236758,
 2352300812,
 2790382446,
 2090060603,
 2042392458,
 3037745336,
 144154533,
 1769088580,
 2373860623,
 2008499596,
 2054774456,
 3178794464,
 2158465477,
 2023158072,
 2106210042,
 2075425215,
 2

In [15]:
# 6min
# (B) 引用先の論文の抽出
B_relations = references_df[references_df['CitedPublicationId'].isin(bibmet_id_list)]

# (C) 参考文献に挙げた論文
C_relations = references_df[references_df['CitingPublicationId'].isin(bibmet_id_list)]

# (D) 更に「参考文献に挙げた論文」を参考文献に挙げている論文の抽出
# まず(C)からCitedPublicationIdを取得し、それを引用している論文を探す
C_cited_ids = C_relations['CitedPublicationId'].unique()
D_relations = references_df[references_df['CitedPublicationId'].isin(C_cited_ids)]

B_relations, D_relations

(        CitingPublicationId CitedPublicationId
 1337             2043840923         2068988534
 103              4386376446         2009532800
 2453             4387745815         2336257495
 8645              197392799         2041318199
 8741             1994982975         1986080923
 ...                     ...                ...
 9371565          2105825040         2054405285
 9376548          2107952196         1981275310
 9377951          2108425242         1423339008
 9403936          2119692351         2056373445
 9410049          2122763672         2086028289
 
 [160072 rows x 2 columns],
         CitingPublicationId CitedPublicationId
 2                3117235547         1540739244
 14               3117235547         2004341664
 18               3117235547         2012538600
 29               3117235547         2065534179
 32               3117235547         2072394847
 ...                     ...                ...
 9436443          2134806910         2052288562
 9436452  

In [16]:
D_relations[D_relations['CitedPublicationId'] == 1976880180]

,CitingPublicationId,CitedPublicationId


In [17]:
D_relations[D_relations['CitingPublicationId'] == 2573593301]

,CitingPublicationId,CitedPublicationId


In [18]:
references_df[references_df['CitedPublicationId'] == 55183993]

,CitingPublicationId,CitedPublicationId
0,3117235547,55183993
0,2031878785,55183993
3247108,1914197438,55183993
454530,3138296954,55183993
271815,648156390,55183993
...,...,...
953090,2460761523,55183993
6128321,585472863,55183993
6986859,2044549312,55183993
2359164,3195613568,55183993


In [19]:
references_df[references_df['CitedPublicationId'] == '55183993']

,CitingPublicationId,CitedPublicationId


In [21]:
works_rel_ref_df = pd.concat([B_relations, C_relations, D_relations]).drop_duplicates().reset_index(drop=True)
works_rel_ref_df.rename(columns={'CitingPublicationId': 'source', 'CitedPublicationId': 'target'}, inplace=True)
works_rel_ref_df

,source,target
0,2043840923,2068988534
1,4386376446,2009532800
2,4387745815,2336257495
3,197392799,2041318199
4,1994982975,1986080923
...,...,...
58683278,2134806910,2052288562
58683279,2134806910,2095784025
58683280,2134806910,2100937930
58683281,2134806910,2143114368


In [33]:
random_works_df.set_index('PublicationId', inplace=True)
random_works_df

,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText,AuthorCount
PublicationId,,,,,,,,,,,,,,,
2289614885,NaN,2015,0,On the margin : technology helps carriers choo...,2015-08-01,article,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1.0
588250911,NaN,2008,0,MISCELLANEOUS 日本語表現能力に関する実践的教育-演習「プロゼミII」の実態調査-,2008-03-18,article,NaN,NaN,2,NaN,94,104,False,False,1.0
1997677555,NaN,2009,1,Errors of target detecting using triangulation...,2009-10-23,article,NaN,NaN,NaN,NaN,413,414,False,False,1.0
641542497,NaN,2011,0,Mellan nation och omvärld,2011-01-01,article,NaN,NaN,NaN,NaN,297,301,False,False,1.0
2252829188,NaN,1998,0,"解决砌块建筑“裂,热,漏”问题的探讨",1998-01-01,article,NaN,NaN,NaN,12,18,18,False,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2039546701,NaN,1993,26,Two-Dimensional Contact-Type Image Sensor Usin...,1993-01-01,article,10.1143/jjap.32.458,NaN,32,1S,458,458,False,False,3.0
2044872246,NaN,2001,31,Photoelectrochemistry of Langmuir−Blodgett Fil...,2001-09-07,article,10.1021/jp010250h,NaN,105,39,9406,9412,False,False,2.0
1973947117,NaN,2006,31,Strain relaxation and surface morphology of ni...,2006-03-01,article,10.1016/j.susc.2005.12.038,NaN,600,5,1099,1106,False,False,7.0


In [34]:
random_works_df.to_pickle(OUTPUT_WORKS_DF_PKL)
works_rel_ref_df.to_pickle(OUTPUT_WORKS_REL_REF_DF_PKL)